# Практические задания 14

* Лекция 30 - Файлы JSON и модуль datetime
* Лекция 31 - Файлы CSV и модуль statistics


0. ***Простая запись и чтение файлов csv и json***  
Создайте три списка: Month – с числами от 1 до 12, Income и Outcome – оба с 12 случайными числами от 2000 до 10000. Запишите эти три списка как столбцы в csv файл. Затем прочитайте этот csv-файл как список словарей. Запишите этот список словарей в json-файл.


1. ***Анализ оценок по кварталам***  
Реализуйте программу, которая должна:  
* Прочитать данные из JSON-файла.  
* Для каждого ученика вычислить среднюю и медианную оценку за каждый учебный квартал:  
   * 1 квартал: январь–март  
   * 2 квартал: апрель–май (лето пропускается)  
   * 3 квартал: сентябрь–декабрь  
      * Записать результаты в CSV-файл с колонками: ученик, квартал, средняя оценка, медианная оценка  
*Данные:*  
Файл grades.json содержит список записей следующего формата:


```
[{"name": "Bob", "subject": "Science", "grade": 86, "date": "06-09-2025"},
  {"name": "Diana", "subject": "Science", "grade": 85, "date": "31-01-2025"},
  {"name": "Bob", "subject": "Literature", "grade": 60, "date": "19-07-2025"},
  {"name": "Charlie", "subject": "Literature", "grade": 78, "date": "05-08-2025"},
  {"name": "Ethan", "subject": "Literature", "grade": 69, "date": "08-04-2025"},
  {"name": "Charlie", "subject": "Science", "grade": 63, "date": "24-10-2025"},
  {"name": "Ethan", "subject": "Math", "grade": 80, "date": "30-01-2025"},
  {"name": "Alice", "subject": "Physics", "grade": 90, "date": "15-09-2025"},
  {"name": "Ethan", "subject": "Science", "grade": 63, "date": "18-09-2025"},
  ...
]
```

 Пример вывода (quarterly_report.csv):

```
Name,Quarter,Average grade,Median grade
Physics,Bob,Q4,80.2,78
Physics,Diana,Q1,88.5,88.5
Physics,Ethan,Q2,69,69
```

3. ***Анализ продаж***  
Прочитайте файл sales.csv в список словарей с ключами fruit, store, date, revenue (то есть количество нужно будет умножить на цену). У всех значений нужно будет сделать корректный тип данных. После этого сформируйте два json-файла, один с информацией о фруктах, другой - с информацией о магазинах. Для каждого фрукта нужно будет вычислить суммарные продажи, среднюю продажу, медианную продажу, стандартное отклонение продаж, дату первой продажи, дату последней продажи. Во втором файле такая же информация, но по каждому магазину.

*Данные:*  
Файл `sales.csv` содержит список записей следующего формата:  
```
Fruit,Store,Date,Quantity,Price
Lemon,Rainbow,2023-01-01,120,1.50
Mango,Farmer,2023-01-01,120,4.80
Grape,Farmer,2023-02-01,80,2.00
```
Пример вывода (store_summary.json):  
```
{"Rainbow": 
{"total_revenue": 922.5, 
"average_revenue": 153.75, 
"median_revenue": 180.0, 
"stddev_revenue": 49.84, 
"first_sale_date": "2023-01-01", 
"last_sale_date": "2023-01-07"}, 
"Farmer": 
{"total_revenue": 1580.0, 
"average_revenue": 225.71, 
"median_revenue": 180.0, 
"stddev_revenue": 155.0, 
"first_sale_date": "2023-01-01", 
"last_sale_date": "2023-01-07"},
…
}
```